In [161]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [133]:
pd.options.display.max_columns=None

In [134]:
df_esios=pd.read_csv('../data/datos_esios.csv', sep=';')

In [135]:
df_esios.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2
0,2015-10-01 00:00:00,2196.1,24517.1,8203.3,5355.4,4705.3,56.65,38.56,56.31,58.30
1,2015-10-01 01:00:00,2064.4,23168.6,8286.8,5355.4,4705.3,53.50,31.27,54.25,56.60
2,2015-10-01 02:00:00,1866.5,22437.2,8301.3,5355.4,4705.3,49.69,30.66,49.69,51.58
3,2015-10-01 03:00:00,1660.9,22281.3,8281.3,5355.4,4705.3,49.10,29.14,49.10,49.57
4,2015-10-01 04:00:00,1373.9,21909.7,8247.7,5355.4,4705.3,51.25,29.40,51.25,52.00


In [136]:
df_esios.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2
count,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.000000,26304.00000,26304.000000
mean,5479.286014,28672.990169,4409.936025,4420.339625,4188.581794,48.739281,42.105033,48.46574,48.474287
std,3185.927399,4616.496856,2216.248299,474.038209,469.363759,14.954689,20.799903,15.09524,15.110244
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.00000,0.000000
25%,3018.400000,24734.150000,2316.975000,4125.800000,3797.600000,40.137500,29.740000,40.05000,40.100000
50%,4822.950000,28920.600000,4634.000000,4483.200000,4148.100000,49.550000,39.165000,49.00000,48.950000
75%,7289.250000,32267.650000,6282.750000,4799.700000,4705.300000,59.450000,51.562500,58.90000,58.710000
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.49000,103.610000


In [137]:
df_esios['datetime'].describe()  #tenemos repetidos los datos para el dia de 25 horas.

count                   26304
unique                  26301
top       2015-10-25 02:00:00
freq                        2
Name: datetime, dtype: object

#### 1) Calculamos columna a predecir:

"reservar"

Nuestra clasificación será:

* Si PMI1 > PMD -> 0

* Si PMI1 < PMD -> 1

* Si PMI1 == PMD:

    * Si PMI2 > PMD -> 0
    * Si PMI2 <= PMD -> 1

En la descarga y exploración de datos esios se obtuvo que de los 26.304 datos, hay 3.493 casos en los que el precio del mercado diario coincidió con el del intra, y de esos, 793 horas en las que además el intradiario 2 tampoco subió/bajó con respecto al diario.

In [143]:
df_esios_precio=df_esios[['PRECIO_MD_ESP','PRECIO_I1','PRECIO_I2','PRECIO_MD_FR']]

In [144]:
reserva=np.zeros(len(df_esios_precio['PRECIO_MD_ESP']))
for k,v in df_esios_precio.iterrows():
    a=v['PRECIO_MD_ESP']
    b=v['PRECIO_I1']
    c=v['PRECIO_I2']
    if (a<b) | ((a==b) & (c>b)):
        reserva[k]=0
    else:
        reserva[k]=1  
df_esios['RESERVA']=reserva

In [142]:
df_esios.sample(10)  #comprobamos aleatoriamente que ok

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA
25867,2018-09-12 19:00:00,2087.3,33044.4,6396.8,4087.2,4353.1,78.78,78.78,78.29,79.76,1.0
11781,2017-02-02 20:00:00,12440.6,36469.2,5055.5,4789.7,3226.4,61.23,48.07,54.25,54.23,1.0
1764,2015-12-13 11:00:00,3880.8,27851.1,7691.0,4723.7,4705.3,49.00,42.15,45.50,49.00,1.0
5672,2016-05-24 08:00:00,4049.9,29082.5,1911.0,3651.5,4135.3,37.09,37.09,37.09,37.19,0.0
22260,2018-04-15 12:00:00,6042.0,26771.8,1355.0,4096.9,4354.5,40.00,24.99,37.00,38.00,1.0
6258,2016-06-17 18:00:00,5172.4,29140.4,2175.4,4293.6,4135.3,43.42,31.28,44.20,43.43,0.0
10148,2016-11-26 19:00:00,5186.0,31740.8,6564.4,4457.0,4143.3,62.48,55.83,64.48,65.11,0.0
4725,2016-04-14 21:00:00,5853.2,33232.3,1066.3,4034.8,3797.1,31.53,31.53,31.22,29.63,1.0
2643,2016-01-19 02:00:00,7056.8,24252.1,3839.2,4338.3,3795.7,35.87,35.87,39.00,37.07,0.0
6866,2016-07-13 02:00:00,6724.7,26105.7,3207.4,4829.4,4135.3,31.79,26.05,36.60,36.00,0.0


Creamos columna ACOPLADO_FR:

*Si Precio FR MD == Precio España MD están acoplados (1) si no no (0)*

In [147]:
acoplado=np.zeros(len(df_esios_precio['PRECIO_MD_ESP']))
for k,v in df_esios_precio.iterrows():
    a=v['PRECIO_MD_ESP']
    b=v['PRECIO_MD_FR']
    if (a==b):
        acoplado[k]=1
    else:
        acoplado[k]=0  
df_esios['ACOPLADO_FR']=acoplado

In [148]:
df_esios.sample(10) #comprobamos aleatoriamente que ok

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR
15669,2017-07-14 21:00:00,8585.0,34527.8,6159.1,4809.7,4665.3,49.80,39.10,49.33,49.80,1.0,0.0
15047,2017-06-18 23:00:00,5635.0,30814.7,6339.6,4686.0,3094.1,56.40,37.08,56.40,55.70,1.0,0.0
19322,2017-12-14 01:00:00,10126.8,26477.6,5312.3,4799.7,4353.2,47.39,47.39,44.49,31.00,1.0,1.0
25016,2018-08-08 08:00:00,1938.5,30823.8,6052.3,4794.7,3791.1,65.67,65.06,64.67,64.27,1.0,0.0
7235,2016-07-28 11:00:00,2521.3,35450.8,6403.9,4829.4,4135.3,46.38,44.99,46.69,46.38,0.0,0.0
8334,2016-09-12 06:00:00,4288.9,25995.0,6493.1,4623.9,4023.3,38.12,36.69,39.67,41.56,0.0,0.0
18343,2017-11-03 06:00:00,2989.9,26028.2,6132.1,4463.6,3266.6,56.10,56.10,56.10,58.00,0.0,1.0
127,2015-10-06 07:00:00,12051.5,27822.9,3173.3,5355.4,4705.3,43.00,45.81,47.02,47.67,0.0,0.0
23966,2018-06-25 14:00:00,1141.5,35684.7,4393.9,4799.7,4705.3,64.01,47.14,62.64,64.02,1.0,0.0
12099,2017-02-16 02:00:00,4362.9,23805.4,4874.2,4529.7,3852.8,44.53,44.53,43.53,44.33,1.0,1.0


Sumamos la potencia disponible de carbón. Nos interesa el total de MW disponibles.

In [153]:
df_esios['POT_DISP_CARBON']=[x['POT_DISP_HULLA_ANT'] + x['POT_DISP_HULA_SB'] for _,x in df_esios.iterrows()]

In [154]:
df_esios.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON
0,2015-10-01 00:00:00,2196.1,24517.1,8203.3,5355.4,4705.3,56.65,38.56,56.31,58.30,1.0,0.0,10060.7
1,2015-10-01 01:00:00,2064.4,23168.6,8286.8,5355.4,4705.3,53.50,31.27,54.25,56.60,0.0,0.0,10060.7
2,2015-10-01 02:00:00,1866.5,22437.2,8301.3,5355.4,4705.3,49.69,30.66,49.69,51.58,0.0,0.0,10060.7
3,2015-10-01 03:00:00,1660.9,22281.3,8281.3,5355.4,4705.3,49.10,29.14,49.10,49.57,0.0,0.0,10060.7
4,2015-10-01 04:00:00,1373.9,21909.7,8247.7,5355.4,4705.3,51.25,29.40,51.25,52.00,0.0,0.0,10060.7


Necesitamos el carbón casado el día anterior (ya que el carbón que se va a casar ese día no lo conocemos) así que tendremos que restar 1 día a la fecha para la columna P48_CARBON, creando la columna adicional P48_CARBON_D-1

Aprovechamos esta operación para crear la columna RESULTADO D-1 que es la columna reserva del día anterior.

In [158]:
df_esios_ant= df_esios[['datetime','P48_CARBON','RESERVA']]

In [159]:
df_esios_ant.head()

,datetime,P48_CARBON,RESERVA
0,2015-10-01 00:00:00,8203.3,1.0
1,2015-10-01 01:00:00,8286.8,0.0
2,2015-10-01 02:00:00,8301.3,0.0
3,2015-10-01 03:00:00,8281.3,0.0
4,2015-10-01 04:00:00,8247.7,0.0


In [162]:
dia=timedelta(days=1) 

In [166]:
df_esios_ant['datetime']=pd.to_datetime(df_esios_ant['datetime'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [163]:
df_esios_ant['aux']=[x+dia for x in df_esios_ant['datetime']]

TypeError: must be str, not datetime.timedelta